In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL
from VMD import VMD

from sklearn.svm import SVR
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
np.random.seed(42)
# dates = pd.date_range(start='2000-01-01', periods=100, freq='Q')
# data = pd.DataFrame({
#     'price': np.random.randn(100).cumsum() + 50,
#     'demand': np.random.randn(100).cumsum() + 100
# }, index=dates)
data=pd.read_excel('goldpd.xlsx')
data.set_index('time', inplace=True)
# 
train_size = int(0.8 * len(data))
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

def model1_forecast(series, period=4, vmd_modes=3, lag=2):
    """STL+VMD+ARIMA+SVR"""
    # STL
    stl = STL(series)
    # stl = STL(series, period=period)
    res = stl.fit()
    trend = res.trend
    seasonal = res.seasonal
    residual = res.resid
    detrended = seasonal + residual
    
    # VMD
    alpha = 2000
    tau = 0.0
    K = vmd_modes
    u, _, _ = VMD(detrended.values, alpha, tau, K, DC=0, init=1, tol=1e-7)
    u=u.astype(float)
    # （ARIMA）
    try:
        modelt=ARIMA(trend.dropna(), order=(0, 1, 1))
        resultt = modelt.fit()
        trend_fc = resultt.forecast()[0]
    except:
        trend_fc = trend.iloc[-1]
    
    # VMD
    vmd_preds = []
    for k in range(K):
        comp = u[k, :]
        if len(comp) < lag + 1:
            vmd_preds.append(0)
            continue
        
       
        X, y = [], []
        for t in range(lag, len(comp)):
            X.append(comp[t-lag:t])
            y.append(comp[t])
        X, y = np.array(X), np.array(y)
        
        
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_scaled = scaler.fit_transform(X)

        scaler1 = MinMaxScaler(feature_range=(0, 1))
        y = scaler1.fit_transform(y.reshape(-1, 1)).reshape(-1)


        last_values = scaler.transform(comp[-lag:].reshape(1, -1))
        
        svr= BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100)
        # svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
        svr.fit(X_scaled, y)
        pred = svr.predict(last_values)[0]
        pred = scaler1.inverse_transform(pred.reshape(-1,1))
        vmd_preds.append(pred)
    
    return trend_fc + np.sum(vmd_preds)

def model2_forecast(df, target_cols, max_lags=2):
    """VAR"""
    try:
        model = VAR(df[target_cols])
        results = model.fit(maxlags=max_lags)
        lag_order = max(results.k_ar, 1)
        fc = results.forecast(df[target_cols].values[-lag_order:], 1)
        return fc[0]
    except:
        return df[target_cols].iloc[-1].values


val_start = train_size - 10  # Verify using the last 10 training points
model1_preds, model2_preds = [], []
true_values = []

for i in range(val_start, train_size):
    current_train = data.iloc[:i]

    price_pred = model1_forecast(current_train['closep'])
    demand_pred = model1_forecast(current_train['closed'])
    model1_preds.append([price_pred, demand_pred])
    

    var_pred = model2_forecast(current_train, ['closep', 'closed'])
    model2_preds.append(var_pred)
    
   
    true_values.append(data.iloc[i][['closep', 'closed']].values)


true_values = np.array(true_values)
model1_preds = np.array(model1_preds)
model2_preds = np.array(model2_preds)

mse_model1 = mean_squared_error(true_values, model1_preds, multioutput='raw_values')
mse_model2 = mean_squared_error(true_values, model2_preds, multioutput='raw_values')

weights = {
    'closep': mse_model2[0] / (mse_model1[0] + mse_model2[0]),
    'closed': mse_model2[1] / (mse_model1[1] + mse_model2[1])
}

# testing
final_preds = []
for i in range(len(test_data)):
    current_data = data.iloc[:train_size+i]
    
    # model 1
    price_pred1 = model1_forecast(current_data['closep'])
    demand_pred1 = model1_forecast(current_data['closed'])
    
    # model 2
    var_pred = model2_forecast(current_data, ['closep', 'closed'])
    
    # combinatorial prediction
    final_price = weights['closep']*price_pred1 + (1-weights['closep'])*var_pred[0]
    final_demand = weights['closed']*demand_pred1 + (1-weights['closed'])*var_pred[1]
    
    final_preds.append([final_price, final_demand])

# 
final_preds = pd.DataFrame(final_preds, 
                          columns=['price_pred', 'demand_pred'],
                          index=test_data.index)

# 
print("Price MSE:", mean_squared_error(test_data['closep'], final_preds['price_pred']))
print("Demand MSE:", mean_squared_error(test_data['closed'], final_preds['demand_pred']))

print("Price MAPE:", np.mean(np.abs((test_data['closep'] - final_preds['price_pred']) / test_data['closep'])) * 100)
print("Demand MAPE:", np.mean(np.abs((test_data['closed'] - final_preds['price_pred']) / test_data['closed'])) * 100)
